[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RichmondAlake/agent_memory_course/blob/main/langmem/memory_augmented_agent_with_mongodb.ipynb)


In [12]:
%pip install -qU langmem langgraph langchain_voyageai langgraph-checkpoint-mongodb langgraph-store-mongodb pymongo requests pypdf langchain-community

Note: you may need to restart the kernel to use updated packages.


In [13]:
import getpass
import os

# 安全地获取并设置环境变量的函数
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [14]:
# set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [15]:
# set_env_securely("VOYAGE_API_KEY", "Enter your Voyage API key: ")

In [16]:
# set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API Key: ")

In [33]:
from langchain_openai import AzureChatOpenAI

# 使用 Azure 时建议关闭 tracing，避免 401（与 zero_to_hero 中一致）
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "true"

azure_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),  # 或直接写 "https://xxx.openai.azure.com/"
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version="2025-03-01-preview",  # 或 "2024-12-01-preview"
    azure_deployment="gpt-4o",          # 你的 deployment 名称
    model="gpt-4o",                     # 模型类型，与 deployment 对应
    temperature=0,
)


In [17]:
MONGODB_URI=os.environ["MONGODB_URI"]
DATABASE_NAME="langmem_agent_memory"
PROCEDURAL_MEMORY_COLLECTION_NAME="procedural_memory"
SEMANTIC_MEMORY_COLLECTION_NAME="semantic_memory"
STATE_CHECKPOINT_COLLECTION_NAME="state_checkpoints"

# 为不同类型的记忆定义命名空间
USER_MEMORY_NAMESPACE = ("user_memories",)  # 用户专属记忆
KNOWLEDGE_NAMESPACE = ("agent_memory_survey",)  # 用于 PDF 内容（与我们的存储一致）

In [18]:
from langchain_voyageai import VoyageAIEmbeddings
from langgraph.store.mongodb.base import MongoDBStore, VectorIndexConfig
from pymongo import MongoClient

client = MongoClient(MONGODB_URI)
db = client[DATABASE_NAME]
procedural_collection = db[PROCEDURAL_MEMORY_COLLECTION_NAME]
semantic_collection = db[SEMANTIC_MEMORY_COLLECTION_NAME]

procedural_vector_index_config = VectorIndexConfig(
    dims=1024,
    index_name="procedural_memory_index",
    filters=None,
    fields=None,
    embed=VoyageAIEmbeddings(model="voyage-3-large"),
)

semantic_vector_index_config = VectorIndexConfig(
    dims=1024,
    index_name="semantic_memory_index",
    filters=None,
    fields=None,
    embed=VoyageAIEmbeddings(model="voyage-3-large"),
)

In [19]:
procedural_memory_store = MongoDBStore(
    collection=procedural_collection,
    index_config=procedural_vector_index_config,
    auto_index_timeout=70
)

In [20]:
semantic_memory_store = MongoDBStore(
    collection=semantic_collection,
    index_config=semantic_vector_index_config,
    auto_index_timeout=70,
)

In [21]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.mongodb import MongoDBSaver

checkpointer = MongoDBSaver(
    client=client,
    db_name=DATABASE_NAME, 
    collection_name=STATE_CHECKPOINT_COLLECTION_NAME,
    index_config=procedural_memory_store
)

## 数据导入与配置

语义记忆导入

In [22]:
import sys

# 方法一：将项目根目录加入 Python 路径
project_root = "/Users/binzhou/Demo/agent_memory_demo/part1"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utilities.pdf_chunker import ingest_pdf_and_chunk

url = "https://arxiv.org/pdf/2404.13501"

# 导入并分块 PDF
chunks = ingest_pdf_and_chunk(url)

Loading PDF with LangChain...
Chunking text...
Successfully created 198 chunks


In [23]:
from utilities.pdf_chunker import store_chunks_in_memory

# 将分块存入语义记忆
store_chunks_in_memory(chunks, semantic_memory_store, KNOWLEDGE_NAMESPACE[0])

# 打印第一个分块作为示例
if chunks:
    print("\nFirst chunk example:")
    print(f"Key: {chunks[0]['key']}")
    print(f"Content preview: {chunks[0]['value']['content'][:200]}...")

Storing 198 chunks in procedural memory...
All chunks stored successfully!

First chunk example:
Key: pdf_chunk_0
Content preview: A Survey on the Memory Mechanism of Large
Language Model based Agents
Zeyu Zhang1, Xiaohe Bo1, Chen Ma1, Rui Li1, Xu Chen1, Quanyu Dai2,
Jieming Zhu2, Zhenhua Dong2, Ji-Rong Wen1
1Gaoling School of Ar...


In [24]:
procedural_memory_store.put(("instructions",), key="agent_instructions", value={"prompt": "Write good paper summaries."})

In [25]:
def prompt(state):
    # 获取流程指令
    item = procedural_memory_store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    
    # 获取用户查询以进行语义检索
    user_query = state["messages"][-1].content
    
    # 在语义记忆中检索相关内容
    knowledge_items = []
    try:
        print("Searching semantic memory for relevant content")
        knowledge_items = semantic_memory_store.search(KNOWLEDGE_NAMESPACE, query=user_query, limit=5)
        print(knowledge_items)
    except Exception as e:
        print(f"Warning: Could not search semantic memory: {e}")
    
    # 用指令和相关知识构建系统内容
    system_content = f"## Instructions\n\n{instructions}\n\n"
    
    if knowledge_items:
        system_content += "## Relevant Knowledge from Agent Memory Research:\n"
        for item in knowledge_items:
            content = item.value.get('content', str(item.value))[:500]  # 限制内容长度
            system_content += f"- {content}...\n"
        system_content += "\nUse this knowledge to provide informed, accurate responses.\n\n"
    
    sys_prompt = {"role": "system", "content": system_content}
    return [sys_prompt] + state['messages']


In [26]:
def simple_prompt(state):
    """仅包含流程指令的简单提示"""
    item = procedural_memory_store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}\n\nYou have access to search tools for knowledge retrieval when needed."}
    return [sys_prompt] + state['messages']

In [27]:
from langmem import create_manage_memory_tool, create_search_memory_tool

# 🚨 tools 参数可转为工具箱记忆（一种流程记忆，用于存储工具及其描述，可使用 BigTools）
# 为不同命名空间创建记忆管理工具
memory_tools = [
    create_manage_memory_tool(USER_MEMORY_NAMESPACE),    # 用户专属记忆
    create_search_memory_tool(USER_MEMORY_NAMESPACE),    # 检索用户记忆
    create_search_memory_tool(KNOWLEDGE_NAMESPACE),      # 检索 PDF 知识库
]

使用记忆工具的智能体 

In [34]:
agent = create_react_agent(
    azure_llm,
    prompt=prompt,  # 从数据库（流程记忆）获取的智能体提示
    tools=memory_tools,
    store= procedural_memory_store,  # 存储语义知识
    checkpointer=checkpointer,  # 在数据库中存储智能体状态（流程记忆）
)

/var/folders/0c/6xdkz3fs2m931gvzk6gz44mh0000gp/T/ipykernel_28142/3022040874.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [35]:
def chat_with_agent(agent, query, thread_id, user_id=None, return_full_result=False):
    """
    增强版对话函数：支持用户专属记忆，并可返回完整结果。

    参数:
        agent: 要调用的智能体
        query: 用户消息文本
        thread_id: 会话线程 ID，用于保持对话连续性
        user_id: 可选用户 ID，用于个性化记忆
        return_full_result: 为 True 时返回完整结果状态；为 False 时仅返回最后一条消息内容

    返回:
        return_full_result=True 时：包含所有消息的完整结果状态
        return_full_result=False 时：仅最后一条消息的内容
    """
    config = {"configurable": {"thread_id": thread_id}}
    if user_id:
        config["configurable"]["user_id"] = user_id
    
    result_state = agent.invoke(
        {"messages": [{"role": "user", "content": query}]}, 
        config=config
    )
    
    if return_full_result:
        return result_state
    else:
        return result_state["messages"][-1].content



In [36]:

response = chat_with_agent(
    agent=agent, 
    query="Who are the authors of the paper A survey on the memory mechanism of large language models?",
    thread_id="10",
    user_id="user-123",
    return_full_result=True
)

Searching semantic memory for relevant content
[Item(namespace=['agent_memory_survey'], key='pdf_chunk_0', value={'content': 'A Survey on the Memory Mechanism of Large\nLanguage Model based Agents\nZeyu Zhang1, Xiaohe Bo1, Chen Ma1, Rui Li1, Xu Chen1, Quanyu Dai2,\nJieming Zhu2, Zhenhua Dong2, Ji-Rong Wen1\n1Gaoling School of Artificial Intelligence, Renmin University of China, Beijing, China\n2Huawei Noah’s Ark Lab, China\nzeyuzhang@ruc.edu.cn, xu.chen@ruc.edu.cn\nAbstract\nLarge language model (LLM) based agents have recently attracted much attention\nfrom the research and industry communities. Compared with original LLMs, LLM-\nbased agents are featured in their self-evolving capability, which is the basis for\nsolving real-world problems that need long-term and complex agent-environment\ninteractions. The key component to support agent-environment interactions is the\nmemory of the agents. While previous studies have proposed many promising mem-\nory mechanisms, they are scattered 

In [37]:
print(response['messages'][-1].content)

The authors of the paper "A Survey on the Memory Mechanism of Large Language Models" are:

- **Zeyu Zhang**
- **Xiaohe Bo**
- **Chen Ma**
- **Rui Li**
- **Xu Chen**
- **Quanyu Dai**
- **Jieming Zhu**
- **Zhenhua Dong**
- **Ji-Rong Wen**

They are affiliated with the Gaoling School of Artificial Intelligence, Renmin University of China, and Huawei Noah’s Ark Lab, China.


In [40]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer(azure_llm)

理解并解释轨迹

In [41]:
current_prompt = procedural_memory_store.get(("instructions",), key="agent_instructions").value["prompt"]
feedback = {"request": "Always respond with sentences starting with, according to the paper in question..."}

optimizer_result = optimizer.invoke({"prompt": current_prompt, "trajectories": [(response["messages"], feedback)]})

In [42]:
print(f"Current prompt: {current_prompt}")
print(f"Optimizer result: {optimizer_result}")


Current prompt: Write good paper summaries.
Optimizer result: Write good paper summaries. Always begin responses with the phrase: 'According to the paper in question...' Ensure consistency in factual information by cross-referencing multiple sources or explicitly stating when there is conflicting information. If uncertain about any information, include a fallback statement such as: 'There appear to be conflicting sources regarding the authors of this paper.'


In [43]:
procedural_memory_store.put(("instructions",), key="agent_instructions", value={"prompt": optimizer_result})


In [44]:
response = chat_with_agent(
    agent,
    "What is the main point of the paper?",
    "0",
    "user-123",
    return_full_result=True
)

Searching semantic memory for relevant content
[Item(namespace=['agent_memory_survey'], key='pdf_chunk_22', value={'content': 'which, however, provide different taxonomies and understandings on LLMs. Following these surveys,\npeople dive into specific aspects of LLMs and review the corresponding milestone studies and key\ntechnologies. These aspects can be classified into four categories including the fundamental problems,\nevaluation, applications, and challenges of LLMs.\nFundamental problems. The surveys in this category aim to summarize techniques that can\nbe leveraged to tackle fundamental problems of LLMs. Specifically, Zhang et al. [8] provide a\ncomprehensive survey on the methods of supervised fine-tuning, which is a key technique for better\ntraining LLMs. Shen et al. [9], Wang et al.[10] and Liu et al. [11] present surveys on the alignment of\nLLMs, which is a key requirement for LLMs to produce outputs consistent with human values. Gao\net al. [12] propose a survey on the 

In [46]:
print(response['messages'][-1].content)

According to the paper in question, the main point appears to focus on the exploration of large language models (LLMs), their fundamental problems, evaluation methods, applications, and challenges. It delves into the design and implementation of memory modules in LLM-based agents, discussing their importance in enhancing reasoning capabilities, storing information in user-friendly formats, and addressing issues like catastrophic forgetting during fine-tuning. Additionally, the paper highlights the applications of memory-enhanced agents and discusses their limitations and future directions. If you have a specific paper in mind, please provide more details for a precise summary.


In [47]:
def eager_prompt(state):
    """使用主动检索的提示（无需 store 参数）"""
    # 获取流程指令
    item = procedural_memory_store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    
    # 获取用户查询以进行语义检索
    user_query = state["messages"][-1].content
    
    # 直接访问语义记忆（无需 store 参数）
    knowledge_items = []
    try:
        knowledge_items = semantic_memory_store.search(KNOWLEDGE_NAMESPACE, query=user_query, limit=3)
    except Exception as e:
        print(f"Warning: Could not search semantic memory: {e}")
    
    # 构建系统内容
    system_content = f"## Instructions\n\n{instructions}\n\n"
    
    if knowledge_items:
        system_content += "## Relevant Knowledge:\n"
        for item in knowledge_items:
            content = item.value.get('content', str(item.value))[:400]
            system_content += f"- {content}...\n"
        system_content += "\n"
    
    sys_prompt = {"role": "system", "content": system_content}
    return [sys_prompt] + state['messages']




In [48]:
# 创建使用主动检索的智能体（语义记忆无需 store 参数）
eager_agent = create_react_agent(
    azure_llm,
    prompt=eager_prompt,  # 自动注入知识
    tools=[],  # 无需检索工具
    store=procedural_memory_store,  # store 中仅含流程记忆
    checkpointer=checkpointer,
)

/var/folders/0c/6xdkz3fs2m931gvzk6gz44mh0000gp/T/ipykernel_28142/3265235424.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  eager_agent = create_react_agent(


In [49]:

print("\n=== Approach 2: Eager Retrieval (Always includes relevant knowledge) ===")
query = "Who are the authors of the paper?"
response2 = chat_with_agent(eager_agent, query, "eager-test", return_full_result=False)
print(f"Response: {response2[:200]}...")





=== Approach 2: Eager Retrieval (Always includes relevant knowledge) ===
Response: According to the paper in question, there appear to be conflicting sources regarding the authors of this paper. If the paper is titled "Communicative agents for software development," the authors are ...


In [50]:
# 安装 BigTool
%pip install -q langgraph-bigtool


Note: you may need to restart the kernel to use updated packages.


In [51]:
import uuid
from typing import Literal, Dict, Any
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph_bigtool import create_agent
from langgraph.prebuilt import InjectedStore
from langgraph.store.base import BaseStore
from typing_extensions import Annotated

# 定义可用的记忆命名空间及其用途
MEMORY_NAMESPACES = {
    "user_memories": "存储与检索用户专属信息、偏好及个人数据",
    "agent_memory_survey": "检索关于智能体记忆机制的综合研究论文",
    "conversations": "存储与检索对话历史与上下文",
    "user_preferences": "管理用户设置、偏好与配置",
    "project_knowledge": "存储与检索项目专属知识与文档",
    "research_papers": "通用研究论文的存储与检索",
}

def create_dynamic_memory_tool_registry() -> Dict[str, Any]:
    """
    创建可动态检索的记忆工具注册表。
    相比将所有工具放在数组中，这种方式更具可扩展性。
    """
    tool_registry = {}
    
    # 为每个命名空间创建检索和管理工具
    for namespace, description in MEMORY_NAMESPACES.items():
        namespace_tuple = (namespace,)
        
        # 该命名空间的检索工具
        search_tool = create_search_memory_tool(namespace_tuple)
        search_tool.description = f"Search {description.lower()}"
        search_id = f"search_{namespace}"
        tool_registry[search_id] = search_tool
        
        # 该命名空间的管理工具
        manage_tool = create_manage_memory_tool(namespace_tuple)
        manage_tool.description = f"Store/update information in {description.lower()}"
        manage_id = f"manage_{namespace}"
        tool_registry[manage_id] = manage_tool
    
    return tool_registry

# 创建工具注册表
memory_tool_registry = create_dynamic_memory_tool_registry()

print(f"Created {len(memory_tool_registry)} memory tools:")
for tool_id, tool in memory_tool_registry.items():
    print(f"  - {tool_id}: {tool.description}")


Created 12 memory tools:
  - search_user_memories: Search store and retrieve user-specific information, preferences, and personal data
  - manage_user_memories: Store/update information in store and retrieve user-specific information, preferences, and personal data
  - search_agent_memory_survey: Search search the comprehensive research paper about agent memory mechanisms
  - manage_agent_memory_survey: Store/update information in search the comprehensive research paper about agent memory mechanisms
  - search_conversations: Search store and retrieve conversation history and context
  - manage_conversations: Store/update information in store and retrieve conversation history and context
  - search_user_preferences: Search manage user settings, preferences, and configuration
  - manage_user_preferences: Store/update information in manage user settings, preferences, and configuration
  - search_project_knowledge: Search store and retrieve project-specific knowledge and documentation
  - 

In [53]:
# 在 store 中为语义检索建立记忆工具索引
from langchain_voyageai import VoyageAIEmbeddings

# 为工具索引创建独立的 store
tool_store = MongoDBStore(
    collection=db["memory_tools"],
    index_config=VectorIndexConfig(
        dims=1024,
        index_name="memory_tools_index", 
        embed=VoyageAIEmbeddings(model="voyage-3-large"),
        filters=None,
        fields=None,
        auto_index_timeout=70
    )
)

# 用每个工具的描述建立索引以支持语义检索
for tool_id, tool in memory_tool_registry.items():
    tool_store.put(
        ("memory_tools",),
        tool_id,
        {
            "name": tool.name,
            "description": tool.description,
            "namespace": tool_id.split("_", 1)[1],  # 从 ID 提取命名空间
            "operation": tool_id.split("_", 1)[0],  # 提取操作类型（search/manage）
        },
    )

print("Memory tools indexed for semantic retrieval")


Memory tools indexed for semantic retrieval


In [54]:
def retrieve_memory_tools(
    query: str,
    operation_type: Literal["search", "manage", "both"] = "both",
    *,
    store: Annotated[BaseStore, InjectedStore],
) -> list[str]:
    """
    根据查询与操作类型智能检索记忆工具。

    参数:
        query: 用户查询，用于确定相关记忆命名空间
        operation_type: 要返回检索类工具、管理类工具或两者
        store: 注入的工具 store，用于搜索

    返回:
        与查询相关的工具 ID 列表
    """
    # 根据查询检索相关工具
    results = store.search(("memory_tools",), query=query, limit=6)
    
    # 按操作类型过滤
    tool_ids = []
    for result in results:
        tool_id = result.key
        tool_data = result.value
        
        if operation_type == "both":
            tool_ids.append(tool_id)
        elif operation_type == "search" and tool_data["operation"] == "search":
            tool_ids.append(tool_id)
        elif operation_type == "manage" and tool_data["operation"] == "manage":
            tool_ids.append(tool_id)
    
    # 对研究类问题确保包含研究论文检索工具
    research_indicators = ["paper", "research", "author", "study", "survey", "memory mechanism"]
    if any(indicator in query.lower() for indicator in research_indicators):
        if "search_agent_memory_survey" not in tool_ids:
            tool_ids.insert(0, "search_agent_memory_survey")
    
    # 限制数量以免给 LLM 过多工具
    return tool_ids[:4]

print("Custom tool retrieval function created")


Custom tool retrieval function created
